In [10]:
import eikon as ek
ek.set_app_key('9a430e13c8a944e9894743c7bd608b7e41a432be')

In [11]:
import eikon as ek  # the Eikon Python wrapper package
import numpy as np  # NumPy
import pandas as pd  # pandas
import cufflinks as cf  # Cufflinks
from sklearn.svm import SVC  # sckikit-learn
from sklearn.model_selection import train_test_split
import configparser as cp
from collections import Counter
from datetime import datetime
import statistics
import datetime as dt
from scipy.stats import ttest_1samp

# S&P 1500 returns

In [12]:
returns,err = ek.get_data(['0#.SPX','0#.SPMIDSM'],
                         ['TR.RecEstValue','TR.RecEstValue.analystcode','TR.RecEstValue.brokername',
             'TR.RecEstValue.date'],
                      {'frq': 'FY', 'SDate':'2010-01-01', 'EDate':'2020-12-31'}) 
returns

,Instrument,Standard Rec (1-5) - Broker Estimate,Analyst Code,Broker Name,Date
0,POOL.OQ,3,4909,LONGBOW RESEARCH,2009-04-03T00:00:00Z
1,POOL.OQ,1,FK6,"MORGAN KEEGAN & COMPANY, INC.",2009-02-20T00:00:00Z
2,POOL.OQ,3,45L4,HILLIARD LYONS,2009-02-23T00:00:00Z
3,POOL.OQ,4,4LF5,PIPER SANDLER COMPANIES,2010-01-05T00:00:00Z
4,POOL.OQ,1,,Permission Denied 104848,2010-02-09T00:00:00Z
...,...,...,...,...,...
188507,ACA.N,2,6TF7,GABELLI & COMPANY,2018-11-19T00:00:00Z
188508,ACA.N,2,<NA>,SUSQUEHANNA FINANCIAL GROUP LLLP,2018-11-14T00:00:00Z
188509,ACA.N,1,3GVL,"OPPENHEIMER & CO., INC.",2019-03-04T00:00:00Z
188510,ACA.N,1,,Permission Denied 75992,2019-09-18T00:00:00Z


In [13]:
returns['Date'] = pd.to_datetime(returns['Date']).dt.date
returns['Date']=pd.to_datetime(returns['Date'],errors='coerce')
returns['Date']=returns['Date'].dt.floor('d')
returns['Date t-1']= returns['Date'] - pd.Timedelta(days=1)
returns['Date t-2']= returns['Date t-1'] - pd.Timedelta(days=1)
returns=returns.dropna()
returns

,Instrument,Standard Rec (1-5) - Broker Estimate,Analyst Code,Broker Name,Date,Date t-1,Date t-2
0,POOL.OQ,3,4909,LONGBOW RESEARCH,2009-04-03,2009-04-02,2009-04-01
1,POOL.OQ,1,FK6,"MORGAN KEEGAN & COMPANY, INC.",2009-02-20,2009-02-19,2009-02-18
2,POOL.OQ,3,45L4,HILLIARD LYONS,2009-02-23,2009-02-22,2009-02-21
3,POOL.OQ,4,4LF5,PIPER SANDLER COMPANIES,2010-01-05,2010-01-04,2010-01-03
4,POOL.OQ,1,,Permission Denied 104848,2010-02-09,2010-02-08,2010-02-07
...,...,...,...,...,...,...,...
188506,ACA.N,2,7SZS,CJS SECURITIES,2018-12-03,2018-12-02,2018-12-01
188507,ACA.N,2,6TF7,GABELLI & COMPANY,2018-11-19,2018-11-18,2018-11-17
188509,ACA.N,1,3GVL,"OPPENHEIMER & CO., INC.",2019-03-04,2019-03-03,2019-03-02
188510,ACA.N,1,,Permission Denied 75992,2019-09-18,2019-09-17,2019-09-16


In [14]:
years = [2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020]   

prices,err = ek.get_data(['0#.SPX','0#.SPMIDSM'],
                         ['TR.PriceClose','TR.PriceClose.date'],
                      {'frq': 'D', 'SDate':'2010-01-01', 'EDate':'2010-12-31'}) 
prices = prices.dropna()
prices

,Instrument,Price Close,Date
0,POOL.OQ,19.34,2010-01-04T00:00:00Z
1,POOL.OQ,19.54,2010-01-05T00:00:00Z
2,POOL.OQ,18.91,2010-01-06T00:00:00Z
3,POOL.OQ,18.79,2010-01-07T00:00:00Z
4,POOL.OQ,18.65,2010-01-08T00:00:00Z
...,...,...,...
312990,THRM.OQ,10.91,2010-12-27T00:00:00Z
312991,THRM.OQ,10.57,2010-12-28T00:00:00Z
312992,THRM.OQ,10.72,2010-12-29T00:00:00Z
312993,THRM.OQ,10.76,2010-12-30T00:00:00Z


In [17]:
for year in years:    
    allprices,err = ek.get_data(['0#.SPX','0#.SPMIDSM'],
                         ['TR.PriceClose','TR.PriceClose.date'],
                      {'frq': 'D', 'SDate':'{}-01-01'.format(year), 'EDate':'{}-12-31'.format(year)}) 
    allprices = allprices.dropna()
    prices=pd.concat([prices,allprices], ignore_index= True)
prices

,Instrument,Price Close,Date
0,POOL.OQ,19.34,2010-01-04T00:00:00Z
1,POOL.OQ,19.54,2010-01-05T00:00:00Z
2,POOL.OQ,18.91,2010-01-06T00:00:00Z
3,POOL.OQ,18.79,2010-01-07T00:00:00Z
4,POOL.OQ,18.65,2010-01-08T00:00:00Z
...,...,...,...
7882391,ACA.N,54.28,2020-12-24T00:00:00Z
7882392,ACA.N,55.22,2020-12-28T00:00:00Z
7882393,ACA.N,53.90,2020-12-29T00:00:00Z
7882394,ACA.N,54.71,2020-12-30T00:00:00Z


In [18]:
prices2009,err2009 = ek.get_data(['0#.SPX','0#.SPMIDSM'],
                         ['TR.PriceClose','TR.PriceClose.date'],
                      {'frq': 'D', 'SDate':'2009-01-01', 'EDate':'2009-12-31'}) 
prices2009[prices2009['Instrument']=='POOL.OQ']

,Instrument,Price Close,Date
0,POOL.OQ,18.08,2009-01-02T00:00:00Z
1,POOL.OQ,18.62,2009-01-05T00:00:00Z
2,POOL.OQ,19.00,2009-01-06T00:00:00Z
3,POOL.OQ,18.39,2009-01-07T00:00:00Z
4,POOL.OQ,18.13,2009-01-08T00:00:00Z
...,...,...,...
247,POOL.OQ,19.00,2009-12-24T00:00:00Z
248,POOL.OQ,19.14,2009-12-28T00:00:00Z
249,POOL.OQ,19.38,2009-12-29T00:00:00Z
250,POOL.OQ,19.40,2009-12-30T00:00:00Z


In [19]:
# filling the price t-2 value for pool.oq directly because 3rd of january 2010 is a weekend and splitting gives timeout

In [20]:
prices['Date'] = pd.to_datetime(prices['Date']).dt.date
prices['Date']=pd.to_datetime(prices['Date'],errors='coerce')
prices['Date']=prices['Date'].dt.floor('d')
prices['Price Close t-1']= prices['Price Close'].shift(1)
prices['Price Close t-2']= prices['Price Close t-1'].shift(1)
prices

,Instrument,Price Close,Date,Price Close t-1,Price Close t-2
0,POOL.OQ,19.34,2010-01-04,NaN,NaN
1,POOL.OQ,19.54,2010-01-05,19.34,NaN
2,POOL.OQ,18.91,2010-01-06,19.54,19.34
3,POOL.OQ,18.79,2010-01-07,18.91,19.54
4,POOL.OQ,18.65,2010-01-08,18.79,18.91
...,...,...,...,...,...
7882391,ACA.N,54.28,2020-12-24,54.54,54.17
7882392,ACA.N,55.22,2020-12-28,54.28,54.54
7882393,ACA.N,53.90,2020-12-29,55.22,54.28
7882394,ACA.N,54.71,2020-12-30,53.90,55.22


In [21]:
prices2009['Date'] = pd.to_datetime(prices2009['Date']).dt.date
prices2009['Date']=pd.to_datetime(prices2009['Date'],errors='coerce')
prices2009['Date']=prices2009['Date'].dt.floor('d')

In [22]:
returnsfinal = pd.merge(returns, prices, how='inner', on=('Instrument','Date'))
returnsfinal

,Instrument,Standard Rec (1-5) - Broker Estimate,Analyst Code,Broker Name,Date,Date t-1,Date t-2,Price Close,Price Close t-1,Price Close t-2
0,POOL.OQ,4,4LF5,PIPER SANDLER COMPANIES,2010-01-05,2010-01-04,2010-01-03,19.54,19.34,NaN
1,POOL.OQ,1,,Permission Denied 104848,2010-02-09,2010-02-08,2010-02-07,19.50,18.58,18.43
2,POOL.OQ,1,,Permission Denied 104848,2010-02-09,2010-02-08,2010-02-07,19.50,18.58,18.43
3,POOL.OQ,3,4LF5,PIPER SANDLER COMPANIES,2010-04-19,2010-04-18,2010-04-17,24.93,23.89,23.63
4,POOL.OQ,3,,Permission Denied 75992,2011-02-02,2011-02-01,2011-01-31,24.35,24.78,24.38
...,...,...,...,...,...,...,...,...,...,...
342732,ACA.N,2,7SZS,CJS SECURITIES,2018-12-03,2018-12-02,2018-12-01,29.33,27.34,26.27
342733,ACA.N,2,479S,D.A. DAVIDSON & COMPANY,2019-11-19,2019-11-18,2019-11-17,38.21,38.28,38.22
342734,ACA.N,1,3GVL,"OPPENHEIMER & CO., INC.",2019-03-04,2019-03-03,2019-03-02,34.88,33.93,33.49
342735,ACA.N,1,,Permission Denied 75992,2019-09-18,2019-09-17,2019-09-16,34.73,34.60,34.64


In [23]:
returnsfinal=returnsfinal.fillna('19.08')
returnsfinal

,Instrument,Standard Rec (1-5) - Broker Estimate,Analyst Code,Broker Name,Date,Date t-1,Date t-2,Price Close,Price Close t-1,Price Close t-2
0,POOL.OQ,4,4LF5,PIPER SANDLER COMPANIES,2010-01-05,2010-01-04,2010-01-03,19.54,19.34,19.08
1,POOL.OQ,1,,Permission Denied 104848,2010-02-09,2010-02-08,2010-02-07,19.50,18.58,18.43
2,POOL.OQ,1,,Permission Denied 104848,2010-02-09,2010-02-08,2010-02-07,19.50,18.58,18.43
3,POOL.OQ,3,4LF5,PIPER SANDLER COMPANIES,2010-04-19,2010-04-18,2010-04-17,24.93,23.89,23.63
4,POOL.OQ,3,,Permission Denied 75992,2011-02-02,2011-02-01,2011-01-31,24.35,24.78,24.38
...,...,...,...,...,...,...,...,...,...,...
342732,ACA.N,2,7SZS,CJS SECURITIES,2018-12-03,2018-12-02,2018-12-01,29.33,27.34,26.27
342733,ACA.N,2,479S,D.A. DAVIDSON & COMPANY,2019-11-19,2019-11-18,2019-11-17,38.21,38.28,38.22
342734,ACA.N,1,3GVL,"OPPENHEIMER & CO., INC.",2019-03-04,2019-03-03,2019-03-02,34.88,33.93,33.49
342735,ACA.N,1,,Permission Denied 75992,2019-09-18,2019-09-17,2019-09-16,34.73,34.60,34.64


# Cleaning the database

In [24]:
returnsfinal = returnsfinal[returnsfinal["Broker Name"].str.contains("Permission Denied")==False]
returnsfinal = returnsfinal.dropna(subset=['Standard Rec (1-5) - Broker Estimate','Analyst Code'])

In [25]:
returnsfinal['Date'] = pd.to_datetime(returnsfinal['Date']).dt.date

In [26]:
start_date = pd.to_datetime("2010-1-1").date()
end_date = pd.to_datetime("2020-12-31").date()
after_start_date = returnsfinal['Date'] >= start_date
before_end_date = returnsfinal['Date'] <= end_date
between_two_dates = after_start_date & before_end_date
filtered = returnsfinal.loc[between_two_dates]
filtered=filtered.drop_duplicates()
filtered

,Instrument,Standard Rec (1-5) - Broker Estimate,Analyst Code,Broker Name,Date,Date t-1,Date t-2,Price Close,Price Close t-1,Price Close t-2
0,POOL.OQ,4,4LF5,PIPER SANDLER COMPANIES,2010-01-05,2010-01-04,2010-01-03,19.54,19.34,19.08
3,POOL.OQ,3,4LF5,PIPER SANDLER COMPANIES,2010-04-19,2010-04-18,2010-04-17,24.93,23.89,23.63
7,POOL.OQ,3,4EUZ,WEDBUSH SECURITIES INC.,2011-04-25,2011-04-24,2011-04-23,28.76,28.77,26.87
31,POOL.OQ,3,2M8D,TRUIST SECURITIES,2012-01-27,2012-01-26,2012-01-25,34.35,34.36,34.33
37,POOL.OQ,3,5Y1B,WILLIAM BLAIR & COMPANY,2011-10-20,2011-10-19,2011-10-18,29.72,28.74,29
...,...,...,...,...,...,...,...,...,...,...
342729,ACA.N,2,6TF7,GABELLI & COMPANY,2018-11-19,2018-11-18,2018-11-17,30.30,31.30,31.33
342731,ACA.N,2,7SZS,CJS SECURITIES,2018-12-03,2018-12-02,2018-12-01,29.33,27.34,26.27
342733,ACA.N,2,479S,D.A. DAVIDSON & COMPANY,2019-11-19,2019-11-18,2019-11-17,38.21,38.28,38.22
342734,ACA.N,1,3GVL,"OPPENHEIMER & CO., INC.",2019-03-04,2019-03-03,2019-03-02,34.88,33.93,33.49


In [27]:
filtered=filtered.drop_duplicates()
filtered['Date']=pd.to_datetime(filtered['Date'],errors='coerce')
filtered['Date']=filtered['Date'].dt.floor('d')
filtered

,Instrument,Standard Rec (1-5) - Broker Estimate,Analyst Code,Broker Name,Date,Date t-1,Date t-2,Price Close,Price Close t-1,Price Close t-2
0,POOL.OQ,4,4LF5,PIPER SANDLER COMPANIES,2010-01-05,2010-01-04,2010-01-03,19.54,19.34,19.08
3,POOL.OQ,3,4LF5,PIPER SANDLER COMPANIES,2010-04-19,2010-04-18,2010-04-17,24.93,23.89,23.63
7,POOL.OQ,3,4EUZ,WEDBUSH SECURITIES INC.,2011-04-25,2011-04-24,2011-04-23,28.76,28.77,26.87
31,POOL.OQ,3,2M8D,TRUIST SECURITIES,2012-01-27,2012-01-26,2012-01-25,34.35,34.36,34.33
37,POOL.OQ,3,5Y1B,WILLIAM BLAIR & COMPANY,2011-10-20,2011-10-19,2011-10-18,29.72,28.74,29
...,...,...,...,...,...,...,...,...,...,...
342729,ACA.N,2,6TF7,GABELLI & COMPANY,2018-11-19,2018-11-18,2018-11-17,30.30,31.30,31.33
342731,ACA.N,2,7SZS,CJS SECURITIES,2018-12-03,2018-12-02,2018-12-01,29.33,27.34,26.27
342733,ACA.N,2,479S,D.A. DAVIDSON & COMPANY,2019-11-19,2019-11-18,2019-11-17,38.21,38.28,38.22
342734,ACA.N,1,3GVL,"OPPENHEIMER & CO., INC.",2019-03-04,2019-03-03,2019-03-02,34.88,33.93,33.49


In [28]:
labels1 = sorted(filtered['Standard Rec (1-5) - Broker Estimate'].unique())
labels1

[0, 1, 2, 3, 4, 5]

In [29]:
filtered = filtered[filtered['Standard Rec (1-5) - Broker Estimate'] != 0]
labels = sorted(filtered['Standard Rec (1-5) - Broker Estimate'].unique())
labels

[1, 2, 3, 4, 5]

In [30]:
filtered[["Price Close", "Price Close t-1" , "Price Close t-2"]] = filtered[["Price Close", "Price Close t-1" , "Price Close t-2"]].apply(pd.to_numeric)
filtered['1+Returni'] = filtered['Price Close']/filtered['Price Close t-1']
filtered['1+Returnj'] = filtered['Price Close t-1']/filtered['Price Close t-2']
filtered

,Instrument,Standard Rec (1-5) - Broker Estimate,Analyst Code,Broker Name,Date,Date t-1,Date t-2,Price Close,Price Close t-1,Price Close t-2,1+Returni,1+Returnj
0,POOL.OQ,4,4LF5,PIPER SANDLER COMPANIES,2010-01-05,2010-01-04,2010-01-03,19.54,19.34,19.08,1.010341,1.013627
3,POOL.OQ,3,4LF5,PIPER SANDLER COMPANIES,2010-04-19,2010-04-18,2010-04-17,24.93,23.89,23.63,1.043533,1.011003
7,POOL.OQ,3,4EUZ,WEDBUSH SECURITIES INC.,2011-04-25,2011-04-24,2011-04-23,28.76,28.77,26.87,0.999652,1.070711
31,POOL.OQ,3,2M8D,TRUIST SECURITIES,2012-01-27,2012-01-26,2012-01-25,34.35,34.36,34.33,0.999709,1.000874
37,POOL.OQ,3,5Y1B,WILLIAM BLAIR & COMPANY,2011-10-20,2011-10-19,2011-10-18,29.72,28.74,29.00,1.034099,0.991034
...,...,...,...,...,...,...,...,...,...,...,...,...
342729,ACA.N,2,6TF7,GABELLI & COMPANY,2018-11-19,2018-11-18,2018-11-17,30.30,31.30,31.33,0.968051,0.999042
342731,ACA.N,2,7SZS,CJS SECURITIES,2018-12-03,2018-12-02,2018-12-01,29.33,27.34,26.27,1.072787,1.040731
342733,ACA.N,2,479S,D.A. DAVIDSON & COMPANY,2019-11-19,2019-11-18,2019-11-17,38.21,38.28,38.22,0.998171,1.001570
342734,ACA.N,1,3GVL,"OPPENHEIMER & CO., INC.",2019-03-04,2019-03-03,2019-03-02,34.88,33.93,33.49,1.027999,1.013138


# Calculating return of each event

In [31]:
def BHRi(row):
    return ((row['1+Returni']*row['1+Returnj'])-1)*100

filtered['BHRi'] = filtered.apply(BHRi, axis=1)
filtered

,Instrument,Standard Rec (1-5) - Broker Estimate,Analyst Code,Broker Name,Date,Date t-1,Date t-2,Price Close,Price Close t-1,Price Close t-2,1+Returni,1+Returnj,BHRi
0,POOL.OQ,4,4LF5,PIPER SANDLER COMPANIES,2010-01-05,2010-01-04,2010-01-03,19.54,19.34,19.08,1.010341,1.013627,2.410901
3,POOL.OQ,3,4LF5,PIPER SANDLER COMPANIES,2010-04-19,2010-04-18,2010-04-17,24.93,23.89,23.63,1.043533,1.011003,5.501481
7,POOL.OQ,3,4EUZ,WEDBUSH SECURITIES INC.,2011-04-25,2011-04-24,2011-04-23,28.76,28.77,26.87,0.999652,1.070711,7.033867
31,POOL.OQ,3,2M8D,TRUIST SECURITIES,2012-01-27,2012-01-26,2012-01-25,34.35,34.36,34.33,0.999709,1.000874,0.058258
37,POOL.OQ,3,5Y1B,WILLIAM BLAIR & COMPANY,2011-10-20,2011-10-19,2011-10-18,29.72,28.74,29.00,1.034099,0.991034,2.482759
...,...,...,...,...,...,...,...,...,...,...,...,...,...
342729,ACA.N,2,6TF7,GABELLI & COMPANY,2018-11-19,2018-11-18,2018-11-17,30.30,31.30,31.33,0.968051,0.999042,-3.287584
342731,ACA.N,2,7SZS,CJS SECURITIES,2018-12-03,2018-12-02,2018-12-01,29.33,27.34,26.27,1.072787,1.040731,11.648268
342733,ACA.N,2,479S,D.A. DAVIDSON & COMPANY,2019-11-19,2019-11-18,2019-11-17,38.21,38.28,38.22,0.998171,1.001570,-0.026164
342734,ACA.N,1,3GVL,"OPPENHEIMER & CO., INC.",2019-03-04,2019-03-03,2019-03-02,34.88,33.93,33.49,1.027999,1.013138,4.150493


# Adjusting by market

In [32]:
market,er2 = ek.get_data('.SPSUP',['TR.PriceClose','TR.PriceClose.date'],
                         {'SDate':'2009-01-01', 'EDate':'2020-12-31','frq':'D'})
market=market.drop_duplicates()
market

,Instrument,Price Close,Date
0,.SPSUP,204.93,2008-12-31T00:00:00Z
1,.SPSUP,211.16,2009-01-02T00:00:00Z
2,.SPSUP,210.20,2009-01-05T00:00:00Z
3,.SPSUP,212.04,2009-01-06T00:00:00Z
4,.SPSUP,205.59,2009-01-07T00:00:00Z
...,...,...,...
3125,.SPSUP,846.95,2020-12-24T00:00:00Z
3127,.SPSUP,853.64,2020-12-28T00:00:00Z
3128,.SPSUP,851.00,2020-12-29T00:00:00Z
3129,.SPSUP,852.67,2020-12-30T00:00:00Z


In [33]:
market['Date'] = pd.to_datetime(market['Date']).dt.date
market['Date']=pd.to_datetime(market['Date'],errors='coerce')
market['Date']=market['Date'].dt.floor('d')
market=market.dropna()
market['Date t-1']= market['Date'] - pd.Timedelta(days=1)
market['Date t-2']= market['Date t-1'] - pd.Timedelta(days=1)
market['Price Close t-1'] = market['Price Close'].shift(1)
market['Price Close t-2'] = market['Price Close t-1'].shift(1)
market

,Instrument,Price Close,Date,Date t-1,Date t-2,Price Close t-1,Price Close t-2
0,.SPSUP,204.93,2008-12-31,2008-12-30,2008-12-29,NaN,NaN
1,.SPSUP,211.16,2009-01-02,2009-01-01,2008-12-31,204.93,NaN
2,.SPSUP,210.20,2009-01-05,2009-01-04,2009-01-03,211.16,204.93
3,.SPSUP,212.04,2009-01-06,2009-01-05,2009-01-04,210.20,211.16
4,.SPSUP,205.59,2009-01-07,2009-01-06,2009-01-05,212.04,210.20
...,...,...,...,...,...,...,...
3125,.SPSUP,846.95,2020-12-24,2020-12-23,2020-12-22,844.16,842.91
3127,.SPSUP,853.64,2020-12-28,2020-12-27,2020-12-26,846.95,844.16
3128,.SPSUP,851.00,2020-12-29,2020-12-28,2020-12-27,853.64,846.95
3129,.SPSUP,852.67,2020-12-30,2020-12-29,2020-12-28,851.00,853.64


In [34]:
market[["Price Close", "Price Close t-1" , "Price Close t-2"]] = market[["Price Close", "Price Close t-1" , "Price Close t-2"]].apply(pd.to_numeric)
market['1+Returni'] = market['Price Close']/market['Price Close t-1']
market['1+Returnj'] = market['Price Close t-1']/market['Price Close t-2']
market

,Instrument,Price Close,Date,Date t-1,Date t-2,Price Close t-1,Price Close t-2,1+Returni,1+Returnj
0,.SPSUP,204.93,2008-12-31,2008-12-30,2008-12-29,NaN,NaN,NaN,NaN
1,.SPSUP,211.16,2009-01-02,2009-01-01,2008-12-31,204.93,NaN,1.030401,NaN
2,.SPSUP,210.20,2009-01-05,2009-01-04,2009-01-03,211.16,204.93,0.995454,1.030401
3,.SPSUP,212.04,2009-01-06,2009-01-05,2009-01-04,210.20,211.16,1.008754,0.995454
4,.SPSUP,205.59,2009-01-07,2009-01-06,2009-01-05,212.04,210.20,0.969581,1.008754
...,...,...,...,...,...,...,...,...,...
3125,.SPSUP,846.95,2020-12-24,2020-12-23,2020-12-22,844.16,842.91,1.003305,1.001483
3127,.SPSUP,853.64,2020-12-28,2020-12-27,2020-12-26,846.95,844.16,1.007899,1.003305
3128,.SPSUP,851.00,2020-12-29,2020-12-28,2020-12-27,853.64,846.95,0.996907,1.007899
3129,.SPSUP,852.67,2020-12-30,2020-12-29,2020-12-28,851.00,853.64,1.001962,0.996907


In [35]:
def BHRm(row):
    return (row['1+Returni']*row['1+Returnj']-1)*100

market['BHRm'] = market.apply(BHRm, axis=1)
market

,Instrument,Price Close,Date,Date t-1,Date t-2,Price Close t-1,Price Close t-2,1+Returni,1+Returnj,BHRm
0,.SPSUP,204.93,2008-12-31,2008-12-30,2008-12-29,NaN,NaN,NaN,NaN,NaN
1,.SPSUP,211.16,2009-01-02,2009-01-01,2008-12-31,204.93,NaN,1.030401,NaN,NaN
2,.SPSUP,210.20,2009-01-05,2009-01-04,2009-01-03,211.16,204.93,0.995454,1.030401,2.571610
3,.SPSUP,212.04,2009-01-06,2009-01-05,2009-01-04,210.20,211.16,1.008754,0.995454,0.416746
4,.SPSUP,205.59,2009-01-07,2009-01-06,2009-01-05,212.04,210.20,0.969581,1.008754,-2.193149
...,...,...,...,...,...,...,...,...,...,...
3125,.SPSUP,846.95,2020-12-24,2020-12-23,2020-12-22,844.16,842.91,1.003305,1.001483,0.479292
3127,.SPSUP,853.64,2020-12-28,2020-12-27,2020-12-26,846.95,844.16,1.007899,1.003305,1.123010
3128,.SPSUP,851.00,2020-12-29,2020-12-28,2020-12-27,853.64,846.95,0.996907,1.007899,0.478186
3129,.SPSUP,852.67,2020-12-30,2020-12-29,2020-12-28,851.00,853.64,1.001962,0.996907,-0.113631


In [36]:
marketcut = market[['Date','BHRm']]
marketcut

,Date,BHRm
0,2008-12-31,NaN
1,2009-01-02,NaN
2,2009-01-05,2.571610
3,2009-01-06,0.416746
4,2009-01-07,-2.193149
...,...,...
3125,2020-12-24,0.479292
3127,2020-12-28,1.123010
3128,2020-12-29,0.478186
3129,2020-12-30,-0.113631


# Merging market-adjusted returns to sorted1

In [37]:
mergedreturns=pd.merge(filtered,marketcut, how ="inner",on="Date")
mergedreturns

,Instrument,Standard Rec (1-5) - Broker Estimate,Analyst Code,Broker Name,Date,Date t-1,Date t-2,Price Close,Price Close t-1,Price Close t-2,1+Returni,1+Returnj,BHRi,BHRm
0,POOL.OQ,4,4LF5,PIPER SANDLER COMPANIES,2010-01-05,2010-01-04,2010-01-03,19.54,19.34,19.080000,1.010341,1.013627,2.410901,1.907453
1,CB.N,3,4DCY,SOLEIL SECURITIES CORP. (HIST),2010-01-05,2010-01-04,2010-01-03,48.16,49.27,16.621459,0.977471,2.964240,189.745926,1.907453
2,PM.N,4,4XEZ,NOMURA SECURITIES INTL (AMERICA),2010-01-05,2010-01-04,2010-01-03,48.92,49.35,46.110000,0.991287,1.070267,6.094123,1.907453
3,VRTX.OQ,5,4JN0,WEDBUSH SECURITIES INC.,2010-01-05,2010-01-04,2010-01-03,42.78,44.24,146.760000,0.966998,0.301445,-70.850368,1.907453
4,ZBH.N,1,3OCS,"WELLS FARGO SECURITIES, LLC",2010-01-05,2010-01-04,2010-01-03,61.92,60.02,54.740000,1.031656,1.096456,13.116551,1.907453
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45667,PSMT.OQ,3,4GWH,SCOTIABANK GBM,2020-05-08,2020-05-07,2020-05-06,54.73,56.12,61.260000,0.975232,0.916095,-10.659484,3.047929
45668,CDAY.N,3,7GRL,CREDIT SUISSE,2018-12-26,2018-12-25,2018-12-24,34.56,31.97,32.130000,1.081013,0.995020,7.563025,2.109917
45669,CCMP.OQ,3,7MI4,BERENBERG,2020-09-11,2020-09-10,2020-09-09,142.07,144.43,144.900000,0.983660,0.996756,-1.953071,-1.694408
45670,NX.N,2,33TA,CJS SECURITIES,2020-09-04,2020-09-03,2020-09-02,18.27,17.20,17.170000,1.062209,1.001747,6.406523,-4.226257


In [38]:
mergedreturns['BHAR'] = mergedreturns['BHRi']- mergedreturns['BHRm']
mergedreturns

,Instrument,Standard Rec (1-5) - Broker Estimate,Analyst Code,Broker Name,Date,Date t-1,Date t-2,Price Close,Price Close t-1,Price Close t-2,1+Returni,1+Returnj,BHRi,BHRm,BHAR
0,POOL.OQ,4,4LF5,PIPER SANDLER COMPANIES,2010-01-05,2010-01-04,2010-01-03,19.54,19.34,19.080000,1.010341,1.013627,2.410901,1.907453,0.503448
1,CB.N,3,4DCY,SOLEIL SECURITIES CORP. (HIST),2010-01-05,2010-01-04,2010-01-03,48.16,49.27,16.621459,0.977471,2.964240,189.745926,1.907453,187.838473
2,PM.N,4,4XEZ,NOMURA SECURITIES INTL (AMERICA),2010-01-05,2010-01-04,2010-01-03,48.92,49.35,46.110000,0.991287,1.070267,6.094123,1.907453,4.186670
3,VRTX.OQ,5,4JN0,WEDBUSH SECURITIES INC.,2010-01-05,2010-01-04,2010-01-03,42.78,44.24,146.760000,0.966998,0.301445,-70.850368,1.907453,-72.757821
4,ZBH.N,1,3OCS,"WELLS FARGO SECURITIES, LLC",2010-01-05,2010-01-04,2010-01-03,61.92,60.02,54.740000,1.031656,1.096456,13.116551,1.907453,11.209098
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45667,PSMT.OQ,3,4GWH,SCOTIABANK GBM,2020-05-08,2020-05-07,2020-05-06,54.73,56.12,61.260000,0.975232,0.916095,-10.659484,3.047929,-13.707413
45668,CDAY.N,3,7GRL,CREDIT SUISSE,2018-12-26,2018-12-25,2018-12-24,34.56,31.97,32.130000,1.081013,0.995020,7.563025,2.109917,5.453108
45669,CCMP.OQ,3,7MI4,BERENBERG,2020-09-11,2020-09-10,2020-09-09,142.07,144.43,144.900000,0.983660,0.996756,-1.953071,-1.694408,-0.258663
45670,NX.N,2,33TA,CJS SECURITIES,2020-09-04,2020-09-03,2020-09-02,18.27,17.20,17.170000,1.062209,1.001747,6.406523,-4.226257,10.632780


In [39]:
sortedreturns = mergedreturns.sort_values(by=['Instrument','Analyst Code'])
sortedreturns

,Instrument,Standard Rec (1-5) - Broker Estimate,Analyst Code,Broker Name,Date,Date t-1,Date t-2,Price Close,Price Close t-1,Price Close t-2,1+Returni,1+Returnj,BHRi,BHRm,BHAR
1655,A.N,2,3QGX,COWEN AND COMPANY,2013-06-13,2013-06-12,2013-06-11,31.814221,31.406621,31.749863,1.012978,0.989189,0.202703,0.651059,-0.448356
4514,A.N,2,3QGX,COWEN AND COMPANY,2011-07-25,2011-07-24,2011-07-23,33.180037,33.530429,33.551882,0.989550,0.999361,-1.108269,-0.510253,-0.598016
18711,A.N,2,3QGX,COWEN AND COMPANY,2016-01-07,2016-01-06,2016-01-05,39.000000,40.730000,40.550000,0.957525,1.004439,-3.822441,-3.668145,-0.154297
22700,A.N,3,3QGX,COWEN AND COMPANY,2014-11-18,2014-11-17,2014-11-16,40.800000,41.240000,41.570000,0.989331,0.992062,-1.852297,0.532954,-2.385251
25038,A.N,3,3QGX,COWEN AND COMPANY,2017-01-04,2017-01-03,2017-01-02,47.100000,46.490000,47.730000,1.013121,0.974021,-1.319925,1.508521,-2.828446
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33340,ZUMZ.OQ,3,7NPK,D.A. DAVIDSON & COMPANY,2018-09-25,2018-09-24,2018-09-23,27.000000,27.600000,28.100000,0.978261,0.982206,-3.914591,-0.487977,-3.426613
21121,ZUMZ.OQ,3,LTC,PIPER SANDLER COMPANIES,2012-01-03,2012-01-02,2012-01-01,26.460000,35.340000,34.150000,0.748727,1.034846,-22.518302,1.050031,-23.568332
41658,ZUMZ.OQ,3,LTC,PIPER SANDLER COMPANIES,2010-10-08,2010-10-07,2010-10-06,24.940000,24.660000,22.800000,1.011354,1.081579,9.385965,0.490802,8.895163
45072,ZUMZ.OQ,2,LTC,PIPER SANDLER COMPANIES,2010-03-04,2010-03-03,2010-03-02,17.620000,16.010000,16.020000,1.100562,0.999376,9.987516,0.413094,9.574421


In [40]:
n = 5
lists = [[[],[],[],[],[]] for i in range (n)]

stocks = sorted(sortedreturns['Instrument'].unique())

ecem = 1
for stock in stocks:
    subdataframe = sortedreturns[sortedreturns['Instrument'] == stock]
    codes = sorted(subdataframe['Analyst Code'].unique())
    for code in codes: 
        auxdf = subdataframe[subdataframe['Analyst Code'] == code]
        
        if len(auxdf) == 1:
            continue
        if len(auxdf) == 2:
            lists[int(auxdf[:1]['Standard Rec (1-5) - Broker Estimate'])-1][int(auxdf[1:2]['Standard Rec (1-5) - Broker Estimate'])-1].append(float(auxdf[1:2]['BHAR']))
        else:
            for i in range(1,len(auxdf)):
                    lists[int(auxdf[i-1:i]['Standard Rec (1-5) - Broker Estimate'])-1][int(auxdf[i:i+1]['Standard Rec (1-5) - Broker Estimate'])-1].append(float(auxdf[i:i+1]['BHAR']))

    ecem += 1
                    
for i in range(len(lists)):
    for j in range(len(lists)):
            lists[i][j] = statistics.mean(lists[i][j])

In [41]:
lists_dataframe = pd.DataFrame(lists, index = labels, columns = labels)

# Table III mean abnormal returns

In [42]:
for i in range(1,6):
    for j in range(1,6):
        if i==j:
            lists_dataframe.loc[i,j]= ['']

lists_dataframe

,1,2,3,4,5
1,,-0.0565365,-1.15214,-2.10115,-2.42242
2,0.956899,,-0.388293,-3.03589,12.8447
3,2.80121,2.7763,,1.04455,-1.63115
4,0.557416,0.884627,1.07909,,-1.25766
5,1.66426,-0.303828,0.391852,-0.208063,


In [43]:
n = 5
lists = [[[],[],[],[],[]] for i in range (n)]

stocks = sorted(sortedreturns['Instrument'].unique())

ecem = 1
for stock in stocks:
    subdataframe = sortedreturns[sortedreturns['Instrument'] == stock]
    codes = sorted(subdataframe['Analyst Code'].unique())
    for code in codes: 
        auxdf = subdataframe[subdataframe['Analyst Code'] == code]
        
        if len(auxdf) == 1:
            continue
        if len(auxdf) == 2:
            lists[int(auxdf[:1]['Standard Rec (1-5) - Broker Estimate'])-1][int(auxdf[1:2]['Standard Rec (1-5) - Broker Estimate'])-1].append(float(auxdf[1:2]['BHAR']))
        else:
            for i in range(1,len(auxdf)):
                    lists[int(auxdf[i-1:i]['Standard Rec (1-5) - Broker Estimate'])-1][int(auxdf[i:i+1]['Standard Rec (1-5) - Broker Estimate'])-1].append(float(auxdf[i:i+1]['BHAR']))

    ecem += 1
                    
for i in range(len(lists)):
    for j in range(len(lists)):
            lists[i][j] = statistics.stdev(lists[i][j])        

In [44]:
lists_dataframe = pd.DataFrame(lists, index = labels, columns = labels)

# Table III standard returns

In [45]:
for i in range(1,6):
    for j in range(1,6):
        if i==j:
            lists_dataframe.loc[i,j]= ['']

lists_dataframe

,1,2,3,4,5
1,,6.36446,37.533,8.36656,8.9678
2,6.53211,,38.5718,11.7539,28.0974
3,37.3356,34.4046,,58.9879,6.40613
4,3.64781,16.0749,21.635,,3.43209
5,3.64562,2.1803,6.59081,9.4655,


In [46]:
n = 5
lists = [[[],[],[],[],[]] for i in range (n)]

stocks = sorted(sortedreturns['Instrument'].unique())

ecem = 1
for stock in stocks:
    subdataframe = sortedreturns[sortedreturns['Instrument'] == stock]
    codes = sorted(subdataframe['Analyst Code'].unique())
    for code in codes: 
        auxdf = subdataframe[subdataframe['Analyst Code'] == code]
        
        if len(auxdf) == 1:
            continue
        if len(auxdf) == 2:
            lists[int(auxdf[:1]['Standard Rec (1-5) - Broker Estimate'])-1][int(auxdf[1:2]['Standard Rec (1-5) - Broker Estimate'])-1].append(float(auxdf[1:2]['BHAR']))
        else:
            for i in range(1,len(auxdf)):
                    lists[int(auxdf[i-1:i]['Standard Rec (1-5) - Broker Estimate'])-1][int(auxdf[i:i+1]['Standard Rec (1-5) - Broker Estimate'])-1].append(float(auxdf[i:i+1]['BHAR']))

    ecem += 1
                    
for i in range(len(lists)):
    for j in range(len(lists)):
            lists[i][j] = ttest_1samp(lists[i][j], popmean=0)      

In [47]:
lists_dataframe = pd.DataFrame(lists, index = labels, columns = labels)
lists_dataframe

,1,2,3,4,5
1,"(4.240205788513086, 2.4764660285168916e-05)","(-0.15840931699337552, 0.8742351377037938)","(-1.324943832207881, 0.18535235022835508)","(-1.3288895437106452, 0.1950062062828341)","(-2.1269563473885507, 0.0374762357921931)"
2,"(3.366120199741745, 0.0008179457702242923)","(4.918548603021804, 9.442697682508117e-07)","(-0.5247282378537769, 0.5998149492301142)","(-3.426592305531167, 0.0007617589563629211)","(1.119782412918657, 0.3136936325226138)"
3,"(3.289272741732218, 0.0010226209120872034)","(4.181404770622569, 2.989950009018903e-05)","(-0.00768843247843305, 0.9938661162156192)","(0.3743853351651342, 0.7082955856657607)","(-3.6278181255531075, 0.0003620132137900526)"
4,"(0.8778157773893275, 0.3865870979695555)","(0.7111656067281817, 0.4779797099255316)","(1.1307941333683007, 0.25867012414457746)","(0.2996904309842642, 0.7650416067649387)","(-1.6792511264859755, 0.10865896134952008)"
5,"(3.5065195991331164, 0.0008833732244736699)","(-0.4406677970549393, 0.669848191894256)","(0.8238235790711911, 0.4110673142394061)","(-0.07290324150336246, 0.9433206996314106)","(-3.2272902014422904, 0.0023928040252423964)"


# Table III T-stat and P-value

In [48]:
for i in range(1,6):
    for j in range(1,6):
        if i==j:
            lists_dataframe.loc[i,j]= ['']

lists_dataframe

,1,2,3,4,5
1,[],"(-0.15840931699337552, 0.8742351377037938)","(-1.324943832207881, 0.18535235022835508)","(-1.3288895437106452, 0.1950062062828341)","(-2.1269563473885507, 0.0374762357921931)"
2,"(3.366120199741745, 0.0008179457702242923)",[],"(-0.5247282378537769, 0.5998149492301142)","(-3.426592305531167, 0.0007617589563629211)","(1.119782412918657, 0.3136936325226138)"
3,"(3.289272741732218, 0.0010226209120872034)","(4.181404770622569, 2.989950009018903e-05)",[],"(0.3743853351651342, 0.7082955856657607)","(-3.6278181255531075, 0.0003620132137900526)"
4,"(0.8778157773893275, 0.3865870979695555)","(0.7111656067281817, 0.4779797099255316)","(1.1307941333683007, 0.25867012414457746)",[],"(-1.6792511264859755, 0.10865896134952008)"
5,"(3.5065195991331164, 0.0008833732244736699)","(-0.4406677970549393, 0.669848191894256)","(0.8238235790711911, 0.4110673142394061)","(-0.07290324150336246, 0.9433206996314106)",[]
